In [1]:
from __future__ import print_function
import os
import sys
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file= 'banglaIsolated_clean.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset_clean']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset_clean']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28) (139000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [3]:
image_size=28
num_labels=10
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28, 1) (139000, 10)
validation set (10000, 28, 28, 1) (10000, 10)
test set (10000, 28, 28, 1) (10000, 10)


In [4]:
log_dir='conv2_Hidden_1layer_log'
def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)== np.argmax(labels, 1))/ predictions.shape[0])
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
    tf.gfile.MakeDirs(log_dir)

In [5]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

sess=tf.InteractiveSession()
graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    with tf.name_scope('layer1_weights'):
        layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
        variable_summaries(layer1_weights)
    with tf.name_scope('layer1_biases'):
        layer1_biases=tf.Variable(tf.zeros([depth]))
        variable_summaries(layer1_biases)
        
    with tf.name_scope('layer2_weights'):
        layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
        variable_summaries(layer2_weights)
    with tf.name_scope('layer2_biases'):
        layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
        variable_summaries(layer2_biases)
    
    with tf.name_scope('layer3_weights'):
        layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
        ##variable_summaries(layer3_weights)
    with tf.name_scope('layer3_biases'):
        layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
        variable_summaries(layer3_biases)
    
    with tf.name_scope('layer4_weights'):
        layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
        variable_summaries(layer4_weights)
    with tf.name_scope('layer4_biases'):
        layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
        variable_summaries(layer4_biases)
    
    #Model
    def model(data):
        with tf.name_scope('conv1'):
            conv=tf.nn.conv2d(data,layer1_weights,[1,2,2,1], padding='SAME')
            hidden=tf.nn.relu(conv+layer1_biases)
        with tf.name_scope('conv2'):
            conv=tf.nn.conv2d(hidden,layer2_weights,[1,2,2,1], padding='SAME')
            hidden=tf.nn.relu(conv+layer2_biases)
        with tf.name_scope('reshaping_data'):
            shape=hidden.get_shape().as_list()
            reshape=tf.reshape(hidden,[shape[0],shape[1]*shape[2]*shape[3]])
            ##image_summ = tf.summary.image('conv2_images', reshape)
        with tf.name_scope('fullyConnected1'):
            hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        with tf.name_scope('fullyConnected2'):
            return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    with tf.name_scope('softmax-crossEntropy'):
        loss=tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
        tf.summary.scalar('cross_entropy',loss)
    
    #optimizer
    
    with tf.name_scope('train'):
        optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [6]:
train_dir=log_dir+'/train'
test_dir=log_dir+'/test'



def dir_maker(path_dir):
    if not (os.path.isdir(path_dir)):
        os.makedirs(path_dir)
    if(os.path.isdir(path_dir)):
        dir_list=os.listdir(path_dir)
        c=0
        for current_dir in dir_list:
            if(current_dir==str(c)):
                print(current_dir)
            c=c+1
        os.makedirs(path_dir+'/'+str(c))
        path_dir=path_dir+'/'+str(c)
        return path_dir
train_dir=dir_maker(train_dir)
test_dir=dir_maker(test_dir)
print(os.listdir(train_dir))
print(os.listdir(test_dir))

[]
[]


In [7]:
num_steps=10001

with tf.Session(graph=graph)as session:
    merged = tf.summary.merge_all()
    
    train_writer = tf.summary.FileWriter(train_dir,sess.graph)
    test_writer = tf.summary.FileWriter(test_dir)
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _,summary, l, predictions =session.run([optimizer,merged,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 0.378617
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 50: 0.109442
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 100: 0.148380
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 150: 5.883938
Minibatch Accuracy: 6.2%
Validation accuracy: 1.2%
Minibatch loss a step 200: 20.452019
Minibatch Accuracy: 0.0%
Validation accuracy: 89.3%
Minibatch loss a step 250: 35.337952
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 300: 233.863037
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 350: 2.607666
Minibatch Accuracy: 0.0%
Validation accuracy: 1.2%
Minibatch loss a step 400: 185.215820
Minibatch Accuracy: 93.8%
Validation accuracy: 1.2%
Minibatch loss a step 450: 3604.144531
Minibatch Accuracy: 93.8%
Validation accuracy: 1.2%
Minibatch loss a step 500: 75199.343750
Minibatch Accuracy: 75.0%
Validation accuracy: 1.2%
Minibatch loss a 

InvalidArgumentError: Nan in summary histogram for: layer1_biases/summaries/histogram
	 [[Node: layer1_biases/summaries/histogram = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](layer1_biases/summaries/histogram/tag, layer1_biases/Variable/read)]]

Caused by op u'layer1_biases/summaries/histogram', defined at:
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages

# convulation using maxpooling operation(nn.max_pool()) of stride of 2 and kernel size 2.

In [21]:
batch_size=16
patch_size=5
depth=16
num_hidden=64

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([image_size//4*image_size//4*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='SAME')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.max_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='SAME')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.max_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='SAME')
        
         #fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        return tf.matmul(hidden, layer4_weights)+ layer4_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    
    optimizer=tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [22]:
num_steps=10001

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

Initialized
Minibatch loss a step 0: 3.003009
Minibatch Accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss a step 50: 2.212623
Minibatch Accuracy: 18.8%
Validation accuracy: 22.5%
Minibatch loss a step 100: 1.873110
Minibatch Accuracy: 56.2%
Validation accuracy: 64.0%
Minibatch loss a step 150: 1.151364
Minibatch Accuracy: 62.5%
Validation accuracy: 74.0%
Minibatch loss a step 200: 0.864053
Minibatch Accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss a step 250: 0.878512
Minibatch Accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss a step 300: 0.861306
Minibatch Accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss a step 350: 0.286721
Minibatch Accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss a step 400: 0.684663
Minibatch Accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss a step 450: 1.018541
Minibatch Accuracy: 68.8%
Validation accuracy: 80.2%
Minibatch loss a step 500: 0.296469
Minibatch Accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch los

# conv with dropout

In [8]:
image_size=28
#create image size function based on input, filter size, padding and stride
#2 convolutions only with 2 pooling
def output_size_pool(input_size, conv_filter_size,pool_filter_size, padding, conv_stride, pool_stride):
    if padding =='same':
        padding=-1.00
    elif padding=='valid':
        padding=0.00
    else:
        return None
    #after convolution 1
    output_1=(((input_size-conv_filter_size-2*padding)/ conv_stride)+1.00)
    #after pool 1
    output_2=(((output_1-pool_filter_size-2*padding)/pool_stride)+1.00)
    #After convolution 2
    output_3=(((output_2-conv_filter_size-2*padding)/conv_stride)+1.00)
    #after pool 2
    output_4=(((output_3-pool_filter_size-2*padding)/pool_stride)+1.00)
    return int(output_4)
final_image_size=output_size_pool(
    input_size=image_size,conv_filter_size=5,pool_filter_size=2,
    padding='valid', conv_stride=1, pool_stride=2)
print(final_image_size)
    

4


In [9]:
batch_size=16
patch_size=5
depth=32
num_hidden=64
beta=0.001

graph=tf.Graph()

with graph.as_default():
    
    #input data
    tf_train_dataset=tf.placeholder(tf.float32, 
                                    shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels=tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    layer1_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,num_channels,depth],stddev=0.1))
    layer1_biases=tf.Variable(tf.zeros([depth]))
    
    layer2_weights=tf.Variable(tf.truncated_normal
                               ([patch_size,patch_size,depth,depth],stddev=0.1))
    layer2_biases=tf.Variable(tf.constant(1.0,shape=[depth]))
    
    final_image_size=output_size_pool(input_size=image_size,
                                      conv_filter_size=5,pool_filter_size=2,padding='valid',
                                      conv_stride=1,pool_stride=2)
    
    
    layer3_weights=tf.Variable(tf.truncated_normal
                               ([final_image_size*final_image_size*depth,num_hidden],stddev=0.1))
    layer3_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    layer4_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_hidden],stddev=0.1))
    layer4_biases=tf.Variable(tf.constant(1.0,shape=[num_hidden]))
    
    
    
    layer5_weights=tf.Variable(tf.truncated_normal
                               ([num_hidden,num_labels],stddev=0.1))
    layer5_biases=tf.Variable(tf.constant(1.0,shape=[num_labels]))
    
    #Model
    def model(data):
        #1st conv layer with pooling
        conv_1=tf.nn.conv2d(data,layer1_weights,[1,1,1,1], padding='VALID')
        hidden_1=tf.nn.relu(conv_1+layer1_biases)
        pool_1=tf.nn.avg_pool(hidden_1,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #2nd conv layer with pooling
        conv_2=tf.nn.conv2d(pool_1,layer2_weights,[1,1,1,1], padding='VALID')
        hidden_2=tf.nn.relu(conv_2+layer2_biases)
        pool_2=tf.nn.avg_pool(hidden_2,[1,2,2,1],[1,2,2,1], padding='VALID')
        
         #1st fully connected layer
        shape=pool_2.get_shape().as_list()
        reshape=tf.reshape(pool_2,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden=tf.nn.relu(tf.matmul(reshape, layer3_weights)+layer3_biases)
        keep_prob=0.5
        hidden_drop=tf.nn.dropout(hidden,keep_prob)
        
        #2nd fully connected layer
        hidden_2=tf.nn.relu(tf.matmul(hidden_drop, layer4_weights)+layer4_biases)
        hidden_2_drop=tf.nn.dropout(hidden_2,keep_prob)
        
        return tf.matmul(hidden_2_drop, layer5_weights)+ layer5_biases
    
    #training computation
    
    logits=model(tf_train_dataset)
    loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    #optimizer
    global_step=tf.Variable(0)
    start_learning_rate=0.05
    learning_rate=tf.train.exponential_decay(start_learning_rate, global_step,100000,0.96,
                                             staircase=True)
    
    optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    #Predictions for the training , validation, and test data
    
    train_prediction= tf.nn.softmax(logits)
    valid_prediction= tf.nn.softmax(model(tf_valid_dataset))
    test_prediction= tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps=50000

with tf.Session(graph=graph)as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset=(step*batch_size)% (train_labels.shape[0]-batch_size)
        batch_data=train_dataset[offset:(offset+batch_size),:,:,:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset : batch_data,tf_train_labels: batch_labels}
        _, l, predictions =session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        if(step%50==0):
            print('Minibatch loss a step %d: %f' %(step,l))
            print('Minibatch Accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%'%accuracy(valid_prediction.eval(),valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss a step 0: 0.470166
Minibatch Accuracy: 18.8%
Validation accuracy: 6.9%
Minibatch loss a step 50: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 100: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 150: nan
Minibatch Accuracy: 75.0%
Validation accuracy: 89.3%
Minibatch loss a step 200: nan
Minibatch Accuracy: 81.2%
Validation accuracy: 89.3%
Minibatch loss a step 250: nan
Minibatch Accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss a step 300: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 350: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 400: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 450: nan
Minibatch Accuracy: 93.8%
Validation accuracy: 89.3%
Minibatch loss a step 500: nan
Minibatch Accuracy: 75.0%
Validation accuracy: 89.3%
Minibatch loss a step 550: nan
Minibatch Accuracy: 75.0%
Valida